# Mapboxgl Python Library for location data visualization

https://github.com/mapbox/mapboxgl-jupyter

## Example with Match-type Color Assignment (Categorical Data)

In [ ]:
import pandas as pd
import os

from mapboxgl.viz import *
from mapboxgl.utils import *

## Set your Mapbox access token.  
Set a  `MAPBOX_ACCESS_TOKEN` environment variable or copy/paste your token  
If you do not have a Mapbox access token, sign up for an account at https://www.mapbox.com/  
If you already have an account, you can grab your token at https://www.mapbox.com/account/

In [ ]:
# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

In [ ]:
# Load data from sample csv
data_url = 'https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/data/cdec.csv'
df = pd.read_csv(data_url)

# Convert Elevation series to float
df['Elevation (feet)'] = df['Elevation (feet)'].astype(float)

# Clean up by dropping null rows
df = df.dropna(axis=1, how='all')

df.head(2)

In [ ]:
# Create geojson data object
cdec_data = df_to_geojson(df.fillna(''),  
                          properties=['CDEC ID', 'CNRFC ID', 'Gage Type', 'Elevation (feet)'], 
                          precision=4) 

In [ ]:
# Assign color stops
category_color_stops = [['reservoir', 'rgb(211,47,47)'],  
                        ['river', 'rgb(81,45,168)'], 
                        ['snow', 'rgb(2,136,209)'], 
                        ['precip', 'rgb(139,195,74)'], 
                        ['temp', 'rgb(255,160,0)']]

# Initialize CircleViz with Categorical Measure Data
viz = CircleViz(cdec_data, 
                access_token=token,
                height='500px',
                label_property='CDEC ID',
                color_property='Gage Type',
                color_default='grey',
                color_function_type='match',
                color_stops=category_color_stops,
                radius=2,
                center=(-121, 38.5),
                zoom=4.5)

# Render map
viz.show() 

## Standard linear interpolation behavior

In [ ]:
# Numeric color stops from ColorBrewer
sample_color_stops = [[0.0, 'rgb(255,255,204)'],
                      [100.0, 'rgb(255,237,160)'],
                      [250.0, 'rgb(254,217,118)'],
                      [500.0, 'rgb(254,178,76)'],
                      [1000.0, 'rgb(253,141,60)'],
                      [2000.0, 'rgb(252,78,42)'],
                      [4000.0, 'rgb(227,26,28)'],
                      [6000.0, 'rgb(189,0,38)'],
                      [10000.0,'rgb(128,0,38)']]

# Select temperature gage records with numeric elevation data
temperature_df = df[df['Gage Type']=='temp']
temperature_data = df_to_geojson(temperature_df, 
                                 properties=['CDEC ID', 'Elevation (feet)'])

# Test CircleViz with interval measure data
viz2 = CircleViz(temperature_data, 
                 access_token=token, 
                 height='400px',
                 color_property='Elevation (feet)', 
                 color_function_type='interpolate', 
                 color_stops=sample_color_stops, 
                 radius=2, 
                 center=(-121, 37.5), 
                 zoom=4.5, 
                 below_layer='waterway-label',
                 legend_key_shape='contiguous-bar',
                 legend_key_borders_on=False)

# Render map
viz2.show()  

## Combination of match-type and interpolate-type color and radius assignment

In [ ]:
# Radius stops for linear interpolation
sample_radius_stops = [[0.0, 1.0],
                       [100.0, 2.0],
                       [500.0, 3.0],
                       [1000.0, 4.0],
                       [5000.0, 5.0],
                       [10000.0, 6.0]]

# Initialize Graduated Circle Visualization 
viz3 = GraduatedCircleViz(cdec_data, 
                          access_token=token, 
                          height='400px',
                          color_function_type='match', 
                          color_stops=category_color_stops, 
                          color_property='Gage Type', 
                          color_default='grey', 
                          opacity=0.5, 
                          radius_property='Elevation (feet)', 
                          radius_stops=sample_radius_stops, 
                          radius_function_type='interpolate', 
                          radius_default=2, 
                          center=(-121, 37.5), 
                          zoom=4.5, 
                          below_layer='waterway-label')

# Render map
viz3.show()

# Use match function for both color and radius

In [ ]:
# Radius stops for linear interpolation
category_radius_stops = [['reservoir', 3],  
                         ['river', 5], 
                         ['snow', 8], 
                         ['precip', 11], 
                         ['temp', 14]]

# Initialize Graduated Circle Visualization 
viz4 = GraduatedCircleViz(cdec_data, 
                          access_token=token, 
                          height='400px',
                          color_function_type='match',
                          color_stops=category_color_stops,
                          color_property='Gage Type',
                          color_default='grey',
                          opacity=0.5,
                          radius_property='Elevation (feet)',
                          radius_stops=category_radius_stops,
                          radius_function_type='match',
                          radius_default=2,
                          center=(-121, 37.5),
                          zoom=4.5,
                          below_layer='waterway-label')

# Render map
viz4.show()